<a href="https://colab.research.google.com/github/XELA3549/IBMEC_IA_FINAN-AS/blob/main/Python/modelos_de_regressao/aula_03_modelos_regressao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
## Importação de pacotes

# Pacotes básicos para manipular dados e criar figuras

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Pacotes utilizados para modelagem

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

Carregando os dados

In [28]:
df = pd.read_csv('aula_03_modelos_regressao.csv', delimiter=';')
df.columns = df.columns.str.strip()
df

FileNotFoundError: [Errno 2] No such file or directory: 'aula_03_modelos_regressao.csv'

Inicialmente, vamos remover a coluna de ano que é igual para todas as linhas



In [ ]:
df.drop(columns=['Year'], inplace=True)

Agora vamos fazer uma análise exploratória simples

In [ ]:
print("Resumo estatístico:")
print(df.describe())

In [ ]:

print("\nValores faltantes por variável:")
print(df.isna().sum())

In [ ]:
# Correlação apenas entre variáveis numéricas
plt.figure(figsize=(14, 10))
sns.heatmap(df.select_dtypes(include='number').corr(), cmap="coolwarm")
plt.title("Matriz de Correlação")
plt.show()

Preparação dos dados para modelagem

In [ ]:
# garantir que não há observações com a target (Life expectancy) ausente

df = df.dropna(subset=["Life expectancy"])

# Salvar nome do país para usar depois
paises = df["Country"]

# Transformar variável categórica 'Status' (em desenvolvimento ou desenvolvido) em dummies (0 ou 1)

df = pd.get_dummies(df, columns=["Status"], drop_first=True)

df

In [ ]:
# Separar target e preditores
X = df.drop(columns=["Life expectancy", "Country"])
y = df["Life expectancy"]

# Manter apenas variáveis numéricas
X_num = X.select_dtypes(include="number")

In [ ]:
# Separar treino e teste
X_train, X_test, y_train, y_test, paises_train, paises_test = train_test_split(
    X_num, y, paises, test_size=0.2, random_state=42
)


In [ ]:
# Imputar valores faltantes com média
imputer = SimpleImputer(strategy="mean")
X_train_imp = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
X_test_imp = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)


In [ ]:
print(X_train_imp.isna().sum())


Agora podemos seguir com a modelagem. Queremos prever a expectativa de vida dos países em 2014 a partir do conjunto de covariáveis dado. Vamos usar o conjunto de treino para que o modelo possa aprender padrões entre variáveis e, em seguida, vamos aplicar o modelos aos dados do conjunto de teste.

In [ ]:
# Modelo linear

modelo_lr = LinearRegression()
modelo_lr.fit(X_train_imp, y_train)

# Predição
y_pred_lr = modelo_lr.predict(X_test_imp)

# Avaliação
rmse = np.sqrt(mean_squared_error(y_test, y_pred_lr))
r2 = r2_score(y_test, y_pred_lr)
print(f"\nRegressão Linear: RMSE = {rmse:.2f} | R² = {r2:.3f}")

# 8) Exibir coeficientes com nomes
coeficientes = modelo_lr.coef_
intercepto = modelo_lr.intercept_

df_coef = pd.DataFrame({
    "Variável": X_train.columns,
    "Coeficiente": coeficientes
})
print("\nCoeficientes da Regressão Linear:")
print(df_coef.sort_values(by="Coeficiente", key=abs, ascending=False))
print(f"\nIntercepto: {intercepto:.2f}")

# 9) Tabela de comparação com país
df_comp_lr = pd.DataFrame({
    "País": paises_test.values,
    "Expectativa de Vida Real": y_test.values,
    "Expectativa de Vida Prevista": y_pred_lr
})

print("\nTabela de comparação (Regressão Linear):")
print(df_comp_lr.sort_values(by="Expectativa de Vida Real", ascending=False).head(10))

Note que o sinal de algumas variáveis parece estar invertido (alcool, sarampo, escolaridade, mortalidade infantil, raquitismo). Dois problemas podem estar ocorrendo:



1.   Multicolinearidade - quando duas ou mais covariáveis são fortemente correlacionadas entre si;
2.  Falta de padronização das variáveis - algumas variáveis tem escala naturalmente pequena (polio, sarampo, pib) e outras naturalmente grande (população).

Vamos, inicialmente, investigar possíveis problemas de multicolinearidade. Em seguida, iremos padronizar as variáveis.



In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Adiciona constante para o modelo (necessário para VIF)
X_vif = sm.add_constant(X_train_imp)

# Calcula o VIF para cada variável
vif_data = pd.DataFrame()
vif_data["Variável"] = X_vif.columns
vif_data["VIF"] = [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]

# Exclui o intercepto da visualização
vif_data = vif_data[vif_data["Variável"] != "const"]

# Ordena por VIF decrescente
vif_data.sort_values(by="VIF", ascending=False, inplace=True)

print(vif_data)

Quanto maior o valor do VIF maior a evidência de multicolineridade. É muito claro que as variáveis 'infant deaths' e 'under-five deaths' e 'thinness 5-9 years' e 'thinness  1-19 years' estão contribuindo com o mesmo tipo de informação. Vamos eliminar uma variável de cada par.

In [ ]:
X_train_imp.columns

In [ ]:
colunas_para_remover = ["under-five deaths", "thinness 5-9 years"]

X_train_imp = X_train_imp.drop(columns=colunas_para_remover)
X_test_imp = X_test_imp.drop(columns=colunas_para_remover)

Também vamos escalar os dados. O StandardScaler transforma as variáveis independentes (covariávies) para média zero e desvio padrão 1. O scaling muda os coeficientes, mas não muda a relação entre as variáveis. Dessa forma, as previsões não serão afetadas. No entanto, para o modelo KNN o scaling é fundamental.

In [29]:
from sklearn.preprocessing import StandardScaler

# Instancia o scaler
scaler = StandardScaler()

# Aplica o scaler nos dados de treino e teste
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train_imp), columns=X_train_imp.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test_imp), columns=X_test_imp.columns)

# Treina novamente o modelo de regressão linear com os dados escalados
modelo_lr_scaled = LinearRegression()
modelo_lr_scaled.fit(X_train_scaled, y_train)

# Predição com os dados escalados
y_pred_lr_scaled = modelo_lr_scaled.predict(X_test_scaled)

# Avaliação do modelo escalado
rmse_scaled = np.sqrt(mean_squared_error(y_test, y_pred_lr_scaled))
r2_scaled = r2_score(y_test, y_pred_lr_scaled)

print(f"\nRegressão Linear (com feature scaling): RMSE = {rmse_scaled:.2f} | R² = {r2_scaled:.3f}")

# Mostrar coeficientes (para facilitar a interpretação, já que os dados estão escalados, os coeficientes indicam impacto padrão)
print(f"Intercepto: {modelo_lr_scaled.intercept_:.2f}")

df_coef_scaled = pd.DataFrame({
    "Variável": X_train_scaled.columns,
    "Coeficiente": modelo_lr_scaled.coef_
})
print("\nCoeficientes da Regressão Linear (dados escalados):")
print(df_coef_scaled.sort_values(by="Coeficiente", key=abs, ascending=False))

# Comparação Real vs Previsto
df_comp_lr = pd.DataFrame({
    "País": X_test_imp.index.map(df.set_index(X.index).loc[:, "Country"]),
    "Real": y_test,
    "Previsto": y_pred_lr
})
print("\nTabela de comparação (Regressão Linear):")
print(df_comp_lr.head())

NameError: name 'X_train_imp' is not defined

Algumas variáveis ainda estão com sinal diferente do esperado pela literatura econômica (e bom senso). Isso significa que ainda há espaço para melhoria do modelo. O proximo modelo é o KNN. Note que voce pode ajustar quantos vizinhos quer usar no modelo. Vamos usar cinco vizinhos. Fique à vontade para testar outras combinações.

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

# Treina o modelo KNN com os dados escalados
modelo_knn = KNeighborsRegressor(n_neighbors=5)  # pode ajustar n_neighbors se quiser
modelo_knn.fit(X_train_scaled, y_train)

# Predição com os dados escalados
y_pred_knn = modelo_knn.predict(X_test_scaled)

# Avaliação do modelo
rmse_knn = np.sqrt(mean_squared_error(y_test, y_pred_knn))
r2_knn = r2_score(y_test, y_pred_knn)

print(f"\nKNN (com feature scaling): RMSE = {rmse_knn:.2f} | R² = {r2_knn:.3f}")

# KNN não tem coeficientes nem intercepto, então só mostraremos a comparação real vs previsto

df_comp_knn = pd.DataFrame({
    "País": X_test_imp.index.map(df.set_index(X.index).loc[:, "Country"]),
    "Real": y_test,
    "Previsto": y_pred_knn
})

print("\nTabela de comparação (KNN):")
print(df_comp_knn.head())

In [ ]:
from sklearn.tree import DecisionTreeRegressor, plot_tree

# Treina o modelo de árvore de decisão com dados escalados
modelo_tree = DecisionTreeRegressor(random_state=42, max_depth=3)  # limite a profundidade para facilitar visualização
modelo_tree.fit(X_train_scaled, y_train)

# Predição com os dados escalados
y_pred_tree = modelo_tree.predict(X_test_scaled)

# Avaliação do modelo
rmse_tree = np.sqrt(mean_squared_error(y_test, y_pred_tree))
r2_tree = r2_score(y_test, y_pred_tree)

print(f"\nÁrvore de Decisão (com feature scaling): RMSE = {rmse_tree:.2f} | R² = {r2_tree:.3f}")

# Comparação Real vs Previsto
df_comp_tree = pd.DataFrame({
    "País": X_test_imp.index.map(df.set_index(X.index).loc[:, "Country"]),
    "Real": y_test,
    "Previsto": y_pred_tree
})

print("\nTabela de comparação (Árvore de Decisão):")
print(df_comp_tree.head())

# Plot da árvore
plt.figure(figsize=(16,8))
plot_tree(modelo_tree, feature_names=X_train_scaled.columns, filled=True, rounded=True, fontsize=10)
plt.title("Árvore de Decisão - Visualização")
plt.show()


Por fim, vamos usar o random forest. Esse modelo é bem mais complexo que os anteriores e, por isso, consegue captar padrões mais sutis entre as variáveis. Abaixo, uma sugestão de configuração do modelo. Voce pode mexer nesses parâmetros para tentar obter resultados melhores.

In [ ]:
# Modelo Random Forest
modelo_rf = RandomForestRegressor(
    n_estimators=100,         # número de árvores da floresta
    max_depth=10 ,            # limita a profundidade (evita overfitting)
    min_samples_split=5,     # exige pelo menos 10 amostras para dividir um nó
    min_samples_leaf=3,       # exige pelo menos 3 amostras por folha
    max_features="sqrt",      # usa raiz quadrada do número total de variáveis em cada split
    bootstrap=True,           # usa amostragem com reposição
    random_state=42           # resultado replicável
)
modelo_rf.fit(X_train_imp, y_train)

# Predição no conjunto de teste
y_pred_rf = modelo_rf.predict(X_test_imp)

# Avaliação
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
r2_rf = r2_score(y_test, y_pred_rf)

print(f"\nRandom Forest: RMSE = {rmse_rf:.2f} | R² = {r2_rf:.3f}")

# Comparação Real vs Previsto
df_comp_tree = pd.DataFrame({
    "País": X_test_imp.index.map(df.set_index(X.index).loc[:, "Country"]),
    "Real": y_test,
    "Previsto": y_pred_rf
})

print("\nTabela de comparação (Árvore de Decisão):")
print(df_comp_tree.head())


Comparativo dos modelos

In [ ]:
resultados_modelos = pd.DataFrame({
    "Modelo": ["Regressão Linear", "KNN", "Árvore de Decisão", "Random Forest"],
    "RMSE": [
        np.sqrt(mean_squared_error(y_test, y_pred_lr_scaled)),
        np.sqrt(mean_squared_error(y_test, y_pred_knn)),
        np.sqrt(mean_squared_error(y_test, y_pred_tree)),
        np.sqrt(mean_squared_error(y_test, y_pred_rf))
    ],
    "R²": [
        r2_score(y_test, y_pred_lr_scaled),
        r2_score(y_test, y_pred_knn),
        r2_score(y_test, y_pred_tree),
        r2_score(y_test, y_pred_rf)
    ]
})

# Ordenar do melhor para o pior RMSE
resultados_modelos.sort_values(by="RMSE", ascending=True, inplace=True)

print("\nComparativo de Modelos:")
print(resultados_modelos)

In [ ]:
plt.figure(figsize=(10, 5))

# RMSE
plt.subplot(1, 2, 1)
sns.barplot(x="RMSE", y="Modelo", data=resultados_modelos, palette="viridis")
plt.title("RMSE por Modelo")

# R²
plt.subplot(1, 2, 2)
sns.barplot(x="R²", y="Modelo", data=resultados_modelos, palette="magma")
plt.title("R² por Modelo")

plt.tight_layout()
plt.show()
